In [1]:
from pathlib import Path
import re
import numpy as np
import pandas as pd



In [2]:

# ⬇️ AJUSTA UNA DE ESTAS OPCIONES

# Opción A: RUTA ABSOLUTA (tu caso)
BONPREU_DIR = Path(r"C:\Users\gerar\Desktop\random\web-scraping\Supermercats\Data\Bonpreu")


# (Opcional) nombre del CSV final
OUT = BONPREU_DIR / "bonpreu_merged_clean.csv"

# Opción B: relativa al cuaderno (si el .ipynb está en Supermercats/)
# BONPREU_DIR = Path.cwd() / "Data" / "Bonpreu"

print("Existe carpeta?:", BONPREU_DIR.exists())
print("Carpeta:", BONPREU_DIR)

# Lista TODO lo que hay
print("\nContenido de la carpeta:")
for p in BONPREU_DIR.iterdir():
    print(" -", p.name)

# Prueba de patrones
cand_all = sorted(BONPREU_DIR.glob("*.csv"))
cand_bp  = sorted(BONPREU_DIR.glob("bonpreu_*.csv"))

print("\n*.csv encontrados:", len(cand_all))
for p in cand_all: print("  ·", p.name)

print("\nbonpreu_*.csv encontrados:", len(cand_bp))
for p in cand_bp: print("  ·", p.name)


Existe carpeta?: True
Carpeta: C:\Users\gerar\Desktop\random\web-scraping\Supermercats\Data\Bonpreu

Contenido de la carpeta:
 - bonpreu_arros.csv
 - bonpreu_llegums-secs_20250920-150513.csv
 - bonpreu_merged_clean.csv
 - bonpreu_pasta-refrigerada_20250920-152250.csv
 - bonpreu_pasta-seca_20250920-150142.csv

*.csv encontrados: 5
  · bonpreu_arros.csv
  · bonpreu_llegums-secs_20250920-150513.csv
  · bonpreu_merged_clean.csv
  · bonpreu_pasta-refrigerada_20250920-152250.csv
  · bonpreu_pasta-seca_20250920-150142.csv

bonpreu_*.csv encontrados: 5
  · bonpreu_arros.csv
  · bonpreu_llegums-secs_20250920-150513.csv
  · bonpreu_merged_clean.csv
  · bonpreu_pasta-refrigerada_20250920-152250.csv
  · bonpreu_pasta-seca_20250920-150142.csv


# 2) Cargar y unir todos los CSV de la carpeta

In [3]:
files = sorted(BONPREU_DIR.glob("bonpreu_*.csv"))
if not files:
    raise FileNotFoundError(f"No se encontraron CSV en {BONPREU_DIR}")

dfs = []
for f in files:
    df = pd.read_csv(f)
    # normalizamos nombres esperados por el scraper
    # (href, name, price, price_per_unit, size)
    cols_lower = {c: c.lower() for c in df.columns}
    df.rename(columns=cols_lower, inplace=True)
    dfs.append(df)

raw = pd.concat(dfs, ignore_index=True)
print(f"CSV combinados: {len(files)}  |  Filas totales: {len(raw)}")
raw.head()


CSV combinados: 5  |  Filas totales: 439


,href,name,price,price_per_unit,size,price (€),price_per_kg
0,https://www.compraonline.bonpreuesclat.cat/pro...,BRILLANTE Arròs rodó tradicional XL,"1,95 €","(4,88 € per quilo)","2 x 0.2kg(4,88 € per quilo)",NaN,NaN
1,https://www.compraonline.bonpreuesclat.cat/pro...,BRILLANTE Arròs integral amb quinoa,"1,49 €","(5,96 € per quilo)","2 x 0.125kg(5,96 € per quilo)",NaN,NaN
2,https://www.compraonline.bonpreuesclat.cat/pro...,MONTSIA Arròs rodó extra,"1,93 €","(1,93 € per quilo)","1kg(1,93 € per quilo)",NaN,NaN
3,https://www.compraonline.bonpreuesclat.cat/pro...,BRILLANTE Arròs integral amb xia,"1,49 €","(5,96 € per quilo)","2 x 0.125kg(5,96 € per quilo)",NaN,NaN
4,https://www.compraonline.bonpreuesclat.cat/pro...,SEGADORS DELTA Arròs extra,"2,15 €","(2,15 € per quilo)","1kg(2,15 € per quilo)",NaN,NaN


# 3) Helpers de parseo (precios y tamaños)

In [4]:
def parse_eur(s):
    """
    Extrae el primer número tipo europeo de un string y lo devuelve como float.
    Ej: '1,95 €' -> 1.95 ; '(4,88 € per quilo)' -> 4.88
    """
    if not isinstance(s, str):
        return np.nan
    m = re.search(r"(\d{1,3}(?:\.\d{3})*|\d+)(?:[,\.]\d+)?", s)
    if not m:
        return np.nan
    num = m.group(0)
    # quita separador de miles y cambia coma por punto
    num = num.replace(".", "").replace(",", ".")
    try:
        return float(num)
    except Exception:
        return np.nan

def kg_from_size(size_str):
    """
    Estima los kg totales a partir del campo 'size'.
    Soporta casos:
      - '1kg', '0.5kg'
      - '500 g'
      - '2 x 0.125kg', '3x500 g'
    Devuelve np.nan si no puede (p.ej. litros).
    """
    if not isinstance(size_str, str):
        return np.nan
    s = size_str.lower().replace("\u00a0", " ")

    # patrón 'n x m unidad'
    m = re.search(r"(\d+)\s*x\s*([\d\.,]+)\s*(kg|g)\b", s)
    if m:
        n = int(m.group(1))
        qty = float(m.group(2).replace(".", "").replace(",", "."))
        unit = m.group(3)
        if unit == "kg":
            return n * qty
        elif unit == "g":
            return n * (qty / 1000.0)

    # patrón simple 'cantidad + unidad'
    m = re.search(r"([\d\.,]+)\s*(kg|g)\b", s)
    if m:
        qty = float(m.group(1).replace(".", "").replace(",", "."))
        unit = m.group(2)
        return qty if unit == "kg" else qty / 1000.0

    return np.nan

def price_per_kg_from_cols(price_str, ppu_str, size_str):
    """
    Primero intenta extraer del campo 'price_per_unit' si dice 'per quilo' o '/kg';
    si no, intenta calcular: price / kg_total (derivado de 'size').
    """
    # 1) desde 'per quilo'
    if isinstance(ppu_str, str) and ("quilo" in ppu_str.lower() or "/kg" in ppu_str.lower()):
        val = parse_eur(ppu_str)
        if pd.notna(val):
            return val

    # 2) calcula desde 'price' y 'size'
    price_val = parse_eur(price_str)
    kg = kg_from_size(size_str)
    if pd.notna(price_val) and pd.notna(kg) and kg > 0:
        return price_val / kg

    return np.nan


# 4) Limpieza y columnas finales

In [5]:
df = raw.copy()

# columnas limpias
df["price (€)"] = df["price"].apply(parse_eur).astype(np.float32)
df["price_per_kg"] = df.apply(
    lambda r: price_per_kg_from_cols(r.get("price"), r.get("price_per_unit"), r.get("size")),
    axis=1
).astype(np.float32)

# nos quedamos con columnas objetivo
out = df[["name", "price (€)", "price_per_kg"]].copy()

# de-duplicar (prefiere el primer registro con ambos precios no nulos)
out["has_ppk"] = out["price_per_kg"].notna().astype(int)
out["has_price"] = out["price (€)"].notna().astype(int)
out.sort_values(["has_ppk","has_price"], ascending=False, inplace=True)
out.drop(columns=["has_ppk", "has_price"], inplace=True)

# si tienes 'href' y/o 'size' en los csv originales y quieres deduplicar mejor:
# out = df.sort_values(["price_per_kg"], na_position="last").drop_duplicates(subset=["name"], keep="first")

out.reset_index(drop=True, inplace=True)
out.head(20)


,name,price (€),price_per_kg
0,BRILLANTE Arròs rodó tradicional XL,1.95,4.88
1,BRILLANTE Arròs integral amb quinoa,1.49,5.96
2,MONTSIA Arròs rodó extra,1.93,1.93
3,BRILLANTE Arròs integral amb xia,1.49,5.96
4,SEGADORS DELTA Arròs extra,2.15,2.15
5,NOMEN Arròs bomba selecció,5.29,5.29
6,BRILLANTE Arròs llarg XL,1.95,4.88
7,NOMEN Arròs llarg,1.39,1.39
8,BONPREU Arròs extra,1.30,1.30
9,NOMEN Arròs rodó,1.89,1.89


In [6]:
out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 439 entries, 0 to 438
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          439 non-null    object 
 1   price (€)     242 non-null    float32
 2   price_per_kg  237 non-null    float32
dtypes: float32(2), object(1)
memory usage: 7.0+ KB


# 5) Tipos y guardados

In [7]:
print(out.dtypes)
out.to_csv(OUT, index=False, encoding="utf-8-sig")
print(f"✅ Guardado: {OUT}  |  filas={len(out)}")


name             object
price (€)       float32
price_per_kg    float32
dtype: object
✅ Guardado: C:\Users\gerar\Desktop\random\web-scraping\Supermercats\Data\Bonpreu\bonpreu_merged_clean.csv  |  filas=439
